In [41]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from prophet import Prophet


In [42]:
data_folder = "data"
filename = "train.csv"
df = pd.read_csv(os.path.join("..", data_folder, filename))

df_subset = df[["warehouse", "orders", "date"]]

df_subset.rename(columns={"date": "ds",
                              "orders": "y"}, inplace=True)

/tmp/ipykernel_9852/3729658863.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.rename(columns={"date": "ds",


In [43]:
warehouses = df_subset["warehouse"].unique().tolist()

In [44]:
filename1 = "test.csv"
df_test = pd.read_csv(os.path.join("..", data_folder, filename1))

df_test_subset = df_test[["date", "warehouse"]].rename(columns={"date": "ds"})

In [45]:
all_predictions = []

for warehouse in warehouses:

    current_df  = df_subset[df_subset["warehouse"] == warehouse].drop(columns=["warehouse"])
    print(current_df.head())
    current_df_test = df_test_subset[df_test_subset["warehouse"] == warehouse].drop(columns=["warehouse"])

    optimal_params = {'changepoint_prior_scale': 0.8,
                  'holidays_prior_scale': 0.001,
                  'seasonality_mode': 'additive',
                  'seasonality_prior_scale': 0.02}

    model = Prophet(**optimal_params)


    model.fit(current_df)

    #future = model.make_future_dataframe(periods=365)

    forecast = model.predict(current_df_test)

    all_predictions += list(forecast["yhat"])

print(all_predictions)

17:18:36 - cmdstanpy - INFO - Chain [1] start processing


        y          ds
0  6895.0  2020-12-05
1  6584.0  2020-12-06
2  7030.0  2020-12-07
3  6550.0  2020-12-08
4  6910.0  2020-12-09


17:18:37 - cmdstanpy - INFO - Chain [1] done processing
17:18:37 - cmdstanpy - INFO - Chain [1] start processing


           y          ds
1193  6447.0  2020-12-05
1194  5338.0  2020-12-06
1195  5621.0  2020-12-07
1196  5792.0  2020-12-08
1197  5894.0  2020-12-09


17:18:37 - cmdstanpy - INFO - Chain [1] done processing
17:18:37 - cmdstanpy - INFO - Chain [1] start processing


...

           y          ds
5557   929.0  2022-02-18
5558   931.0  2022-02-19
5559   945.0  2022-02-23
5560   955.0  2022-02-24
5561  1022.0  2022-02-25


17:18:39 - cmdstanpy - INFO - Chain [1] done processing
17:18:39 - cmdstanpy - INFO - Chain [1] start processing


           y          ds
6186  4623.0  2020-12-05
6187  4197.0  2020-12-06
6188  4452.0  2020-12-07
6189  4714.0  2020-12-08
6190  4450.0  2020-12-09


17:18:40 - cmdstanpy - INFO - Chain [1] done processing


[9811.29013037464, 9780.438280716055, 10032.83639190523, 9752.995986578237, 9765.478297930267, 9969.997548725292, 10812.3288647689, 9769.681963996616, 9754.621928936986, 10022.66083228047, 9757.704748633001, 9783.741824424384, 9999.964792113895, 10851.696258836711, 9815.77736985887, 9804.500692843696, 10073.210723716962, 9805.756282007547, 9826.180313692024, 10033.84044941579, 10874.328725982694, 9824.851262651186, 9798.14903611864, 10050.083550196727, 9765.07214786249, 9767.759033041199, 9958.115951474125, 10782.341573712261, 9718.221020403284, 9679.02731608024, 9921.089183375605, 9629.210495872152, 9628.327964365151, 9818.601434670778, 10646.306176718215, 9589.188839474797, 9560.368982573653, 9815.91060769264, 9540.248905005794, 9557.858575693348, 9768.357790376715, 10617.416750143653, 9582.13202938707, 9574.948048959264, 9851.249490144019, 9594.809221111112, 9629.477441409905, 9854.305457456663, 10714.472436964501, 9686.675750112945, 9683.065739346372, 9958.848302204928, 9697.739526

In [46]:
final_result = pd.concat([df_test["id"], pd.DataFrame({"orders": all_predictions})], axis=1)


#final_result.to_csv("submission.csv", index=False)

In [48]:
final_result.to_csv("submission.csv", index=False)